In [1]:
import pandas as pd

In [2]:
columns_to_keep = [
    "Culmen Length (mm)",
    "Culmen Depth (mm)",
    "Flipper Length (mm)",
    "Body Mass (g)",
    "Sex",
    "Species",
]
penguins = pd.read_csv("penguins.csv")[columns_to_keep].dropna()
penguins["Species"] = penguins["Species"].str.split(" ").str[0]
penguins.head()

,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Species
0,39.1,18.7,181.0,3750.0,MALE,Adelie
1,39.5,17.4,186.0,3800.0,FEMALE,Adelie
2,40.3,18.0,195.0,3250.0,FEMALE,Adelie
4,36.7,19.3,193.0,3450.0,FEMALE,Adelie
5,39.3,20.6,190.0,3650.0,MALE,Adelie


In [3]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
Index: 334 entries, 0 to 343
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Culmen Length (mm)   334 non-null    float64
 1   Culmen Depth (mm)    334 non-null    float64
 2   Flipper Length (mm)  334 non-null    float64
 3   Body Mass (g)        334 non-null    float64
 4   Sex                  334 non-null    object 
 5   Species              334 non-null    object 
dtypes: float64(4), object(2)
memory usage: 18.3+ KB
